In [2]:
import gc

from network_models.soundsream_models_and_utils.ss_encoded_dataset import ss_encoded_dataset_full

data_set= ss_encoded_dataset_full(
    csvPath="/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/allEncs_betterSS_12_1_relwrap_music.pkl", device="cuda")

2023-02-22 14:48:08.618038: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-22 14:48:09.280638: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-22 14:48:09.280701: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-22 14:48:09.280706: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [75]:
import gc
import torch
from torch import nn
import numpy as np
gc.collect()
x = torch.unsqueeze(data_set[0][0], dim=1)

xSize, ySize = 512, 175
kernelSize = 5
first_stride = 1
max_pool_size = 2



enda = int((((xSize - np.floor(kernelSize / 2) * 2) / (max_pool_size*first_stride)) - (np.floor(kernelSize / 2) * 2)*4) // (max_pool_size*4))
endb = int((((ySize - np.floor(kernelSize / 2) * 2) / (max_pool_size*first_stride)) - (np.floor(kernelSize / 2) * 2)*4) // (max_pool_size*4))

def conv_output(x_size, y_size, kernel_size= 5, number_layers = 4, max_size = 2):
    # endx = int((x_size- ((kernel_size-1)*number_layers) - (number_layers*max_size) ) //(max_size*number_layers) )
    # endy = int((y_size- ((kernel_size-1)*number_layers) - (number_layers*max_size) ) //(max_size*number_layers) )
    out = (x_size*y_size - kernel_size) // max_size + 1

    return int(out) if number_layers == 0 else conv_output(out, 1, kernel_size, number_layers -1, max_size)

conv1 = nn.Conv2d(1, 6, kernelSize, stride=1)
conv2 = nn.Conv2d(6, 16, kernelSize, stride=1)
conv3 = nn.Conv2d(16, 32, kernelSize, stride=1)
conv4 = nn.Conv2d(32, 64, kernelSize, stride=1)



pool2d = nn.MaxPool2d(stride=max_pool_size, kernel_size=max_pool_size)
linear1 = nn.Linear(64*conv_output(xSize, ySize), 300)



def forward(x):
    x = conv1(x)
    x = pool2d(x)
    x = conv2(x)
    x = pool2d(x)
    x = conv3(x)
    x = pool2d(x)
    print(x.shape)
    x = conv4(x)
    print(x.shape)
    x = pool2d(x)

    #x = x.view(-1, 32*endb*endb)
    print(x.shape)
    x = x.flatten()
    x = linear1(x)
    return  x

In [76]:
forward(x).shape


torch.Size([1, 32, 60, 18])
torch.Size([1, 64, 56, 14])
torch.Size([1, 64, 28, 7])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x12544 and 179008x300)

In [11]:
torch.unsqueeze(data_set[0][0], dim=1).shape

torch.Size([1, 1, 512, 175])